# Path Tracing Notebook
Implementation of Light Transport Algorithms

********************************************************************************************

In [14]:
import taichi as ti
ti.init(arch=ti.cpu, debug=True, random_seed=36279)

[Taichi] Starting on arch=arm64


In [15]:
import time
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from taichi.math import vec3, cross, normalize

In [16]:
from primitives.primitives import Triangle, Sphere, Primitive
from base.frame import Frame, frame_from_z
from base.materials import Material
from base.bsdf import BSDF
from primitives.ray import Ray
from base.lights import DiffuseAreaLight, UniformLightSampler
from base.camera import PerspectiveCamera

In [17]:
# Constants
MAX_TRIANGLES = 1024
MAX_SPHERES = 256
MAX_LIGHTS = 128
MAX_PRIMITIVES = MAX_TRIANGLES + MAX_SPHERES


# Taichi fields for storing scene data efficiently
num_triangles = ti.field(dtype=ti.i32, shape=())
num_spheres = ti.field(dtype=ti.i32, shape=())
num_lights = ti.field(dtype=ti.i32, shape=())
num_primitives = ti.field(dtype=ti.i32, shape=())

triangles = Triangle.field(shape=(MAX_TRIANGLES,))
spheres = Sphere.field(shape=(MAX_SPHERES,))
lights = DiffuseAreaLight.field(shape=(MAX_LIGHTS,))
primitives = Primitive.field(shape=(MAX_PRIMITIVES,))
materials = Material.field(shape=(MAX_PRIMITIVES,))

# Light sampler field
light_sampler = UniformLightSampler.field(shape=())

In [128]:
cornell_box = "../scenes/cornell-box.pbrt"
volumetric_caustic = "../scenes/volumetric-caustic.pbrt"
door_scene = "../scenes/veach/veach-ajar.pbrt"


file_path = door_scene

In [129]:
import numpy as np
import re

# --- Helper Functions ---

def clean_brackets(s):
    """
    Remove leading and trailing square brackets and quotes from a string.
    E.g. '[ 0.63 0.065 0.05 ]' becomes '0.63 0.065 0.05'
    """
    return s.strip().strip("[]").strip('"')

def parse_value(param_type, raw_value):
    """
    Convert a PBRT parameter string (e.g. "[ 0.63 0.065 0.05 ]") to an appropriate Python value.
    For "texture" and "string" types, return the first token.
    """
    cleaned = clean_brackets(raw_value)
    tokens = re.split(r'\s+', cleaned.strip())
    if not tokens or tokens[0] == "":
        return None
    try:
        if param_type == "float":
            return float(tokens[0])
        elif param_type == "integer":
            return int(tokens[0])
        elif param_type in ["point3", "vector3", "rgb", "spectrum"]:
            if len(tokens) >= 3:
                return tuple(float(tok) for tok in tokens[:3])
            else:
                return None
        elif param_type == "point2":
            if len(tokens) >= 2:
                return tuple(float(tok) for tok in tokens[:2])
            else:
                return None
        elif param_type == "integer[]":
            return list(map(int, tokens))
        elif param_type == "bool":
            return tokens[0].lower() == "true"
        elif param_type in ["string", "texture"]:
            return tokens[0]
        else:
            return cleaned
    except Exception:
        return None

# --- Camera Parsing ---

def parse_camera_from_file(filename):
    """ Reads a PBRT file and extracts Camera parameters into a structured NumPy array. """

    # Set up default camera parameters.
    camera_data = {
        "type": None,             # Camera Type (perspective, orthographic, realistic, spherical)
        "shutteropen": 0.0,
        "shutterclose": 1.0,
        "frameaspectratio": None,
        "screenwindow": None,
        "lensradius": 0.0,
        "focaldistance": 1e30,    # 10^30 (PBRT default)
        "fov": 90.0,              # Perspective camera default
        "mapping": "equalarea",   # Spherical camera default
        "lensfile": "",
        "aperturediameter": 1.0,
        "focusdistance": 10.0,
        "aperture": None
    }

    with open(filename, "r") as file:
        lines = file.readlines()

    current_directive = None  # Tracks active directive (for indented properties)

    for line in lines:
        stripped_line = line.strip()
        if not stripped_line:
            continue

        # When we see a new non-indented directive (and we are in Camera mode), exit the Camera block.
        if current_directive == "Camera" and not line.startswith(" "):
            current_directive = None

        # Process the Camera directive.
        if stripped_line.startswith("Camera"):
            # Example line: Camera "perspective"
            m = re.search(r'Camera\s+"([^"]+)"', stripped_line)
            if m:
                camera_data["type"] = m.group(1)
            current_directive = "Camera"
            continue

        # Process lines within the Camera block.
        if current_directive == "Camera" and stripped_line.startswith('"'):
            # Use regex to extract the quoted parameter declaration (e.g. "float fov")
            m_decl = re.search(r'"([^"]+)"', stripped_line)
            if not m_decl:
                continue
            param_decl = m_decl.group(1)  # e.g., "float fov"
            parts = param_decl.split()
            if len(parts) < 2:
                continue
            ptype, key = parts[0], parts[1]

            # Extract the value between square brackets.
            m_val = re.search(r'\[(.*?)\]', stripped_line)
            if not m_val:
                continue
            raw_value = f"[{m_val.group(1)}]"
            value = parse_value(ptype, raw_value)
            if value is not None:
                camera_data[key] = value
            continue

    # Define a NumPy structured dtype.
    camera_dtype = np.dtype([
        ("type", "U15"),
        ("shutteropen", "f4"),
        ("shutterclose", "f4"),
        ("frameaspectratio", "f4"),
        ("screenwindow", "O"),  # Object (e.g. list of floats)
        ("lensradius", "f4"),
        ("focaldistance", "f4"),
        ("fov", "f4"),
        ("mapping", "U15"),
        ("lensfile", "U50"),
        ("aperturediameter", "f4"),
        ("focusdistance", "f4"),
        ("aperture", "U50"),
    ])

    camera_np = np.array([(camera_data["type"],
                             camera_data["shutteropen"],
                             camera_data["shutterclose"],
                             camera_data["frameaspectratio"] if camera_data["frameaspectratio"] is not None else np.nan,
                             camera_data["screenwindow"],
                             camera_data["lensradius"],
                             camera_data["focaldistance"],
                             camera_data["fov"],
                             camera_data["mapping"],
                             camera_data["lensfile"],
                             camera_data["aperturediameter"],
                             camera_data["focusdistance"],
                             camera_data["aperture"] if camera_data["aperture"] is not None else "")],
                           dtype=camera_dtype)

    return camera_np

# ----------------------------------------------
# Example Usage
# ----------------------------------------------


camera_np = parse_camera_from_file(file_path)

print("Camera Parsed Successfully!")
print("Camera Type:", camera_np["type"][0])
print("FOV:", camera_np["fov"][0])
print("Shutter Open:", camera_np["shutteropen"][0])
print("Shutter Close:", camera_np["shutterclose"][0])
print("Frame Aspect Ratio:", camera_np["frameaspectratio"][0])
print("Lens Radius:", camera_np["lensradius"][0])
print("Focal Distance:", camera_np["focaldistance"][0])


Camera Parsed Successfully!
Camera Type: perspective
FOV: 35.9834
Shutter Open: 0.0
Shutter Close: 1.0
Frame Aspect Ratio: nan
Lens Radius: 0.0
Focal Distance: 1e+30


In [130]:
camera_np['fov'][0]

35.9834

In [131]:
import numpy as np
import re

def parse_value(param_type, raw_value):
    """
    Convert a PBRT parameter string (e.g. "[ 0.63 0.065 0.05 ]") to an appropriate Python value.
    For "texture" and "string" types, return the first token.
    """
    # Remove outer brackets and quotes.
    cleaned = raw_value.strip().strip("[]").strip('"')
    tokens = re.split(r'\s+', cleaned.strip())
    if not tokens or tokens[0] == "":
        return None
    try:
        if param_type == "float":
            return float(tokens[0])
        elif param_type == "integer":
            return int(tokens[0])
        elif param_type in ["point3", "vector", "vector3", "rgb", "spectrum"]:
            if len(tokens) >= 3:
                return np.array([float(tok) for tok in tokens[:3]])
            else:
                return None
        elif param_type == "point":
            # if just a point, assume 3 components (or adjust if needed)
            return np.array([float(tok) for tok in tokens])
        elif param_type == "bool":
            return tokens[0].lower() == "true"
        elif param_type in ["string", "texture"]:
            return tokens[0]
        else:
            return cleaned
    except Exception:
        return None

def parse_light_property_line(line):
    """
    Given a line from a PBRT file (expected to be indented and starting with a quoted declaration),
    extract the parameter type, parameter name, and its parsed value.

    Example line:
        "float coneangle" [ 30.0 ]

    Returns a tuple (ptype, key, value) or (None, None, None) if parsing fails.
    """
    # Use regex to extract the quoted declaration.
    m_decl = re.search(r'"([^"]+)"', line)
    if not m_decl:
        return None, None, None
    declaration = m_decl.group(1)  # e.g., 'float coneangle'
    parts = declaration.split()
    if len(parts) < 2:
        return None, None, None
    ptype, key = parts[0], parts[1]

    # Extract the bracketed value
    m_val = re.search(r'\[([^\]]+)\]', line)
    if not m_val:
        return ptype, key, None
    raw_value = f"[{m_val.group(1)}]"
    value = parse_value(ptype, raw_value)
    return ptype, key, value

def parse_pbrt_lights_and_counts(filename):
    """
    Parses a PBRT file to count and categorize light sources and area lights.

    Returns:
      - A NumPy structured array containing light properties.
      - A dictionary with counts of different light types.
    """
    lights = []
    light_counts = {
        "total_lights": 0,
        "point": 0,
        "spot": 0,
        "distant": 0,
        "infinite": 0,
        "goniometric": 0,
        "projection": 0,
        "area": 0,
    }

    # Pattern to extract values inside brackets.
    bracket_pattern = re.compile(r"\[([^\]]+)\]")

    is_area_light = False

    with open(filename, "r") as f:
        lines = f.readlines()

    i = 0
    while i < len(lines):
        line = lines[i]
        stripped_line = line.strip()
        if not stripped_line:
            i += 1
            continue

        # Detect AttributeBegin/End to flag area lights.
        if stripped_line.startswith("AttributeBegin"):
            is_area_light = True
            i += 1
            continue
        if stripped_line.startswith("AttributeEnd"):
            is_area_light = False
            i += 1
            continue

        # Detect light source directives.
        if stripped_line.startswith("LightSource") or stripped_line.startswith("AreaLightSource"):
            # Extract the light type from the tokens.
            tokens = stripped_line.split()
            if len(tokens) < 2:
                i += 1
                continue
            light_type = tokens[1].strip('"')

            # Update counts
            if is_area_light:
                light_counts["area"] += 1
            elif light_type in light_counts:
                light_counts[light_type] += 1
            light_counts["total_lights"] += 1

            # Initialize default light data.
            light_data = {
                "type": light_type,
                "is_area": is_area_light,
                "from": np.array([0.0, 0.0, 0.0]),
                "to": np.array([0.0, 0.0, 1.0]),
                "I": np.array([1.0, 1.0, 1.0]),
                "L": np.array([1.0, 1.0, 1.0]),
                "scale": 1.0,
                "power": 0.0,
                "illuminance": 0.0,
                "filename": "",
                "coneangle": 30.0,
                "conedeltaangle": 5.0,
                "twosided": False,
                # You can add an extra field to store unknown parameters if desired.
                "extra": {}
            }

            # Process the indented light property lines.
            i += 1
            while i < len(lines) and lines[i].lstrip().startswith('"'):
                ptype, key, value = parse_light_property_line(lines[i])
                if key is None:
                    i += 1
                    continue
                # If the key exists in the default light_data, assign it.
                if key in light_data:
                    light_data[key] = value
                else:
                    # Otherwise, store it in the 'extra' sub-dictionary (or log it).
                    light_data["extra"][key] = value
                    # For debugging: Uncomment the next line to log unknown parameters.
                    # print(f"Unknown parameter '{key}' found in light source.")
                i += 1

            lights.append(light_data)
        else:
            i += 1

    # Convert the list of light dictionaries to a NumPy structured array.
    dtype = np.dtype([
        ("type", "U20"),
        ("is_area", "bool"),
        ("from", "3f4"),
        ("to", "3f4"),
        ("I", "3f4"),
        ("L", "3f4"),
        ("scale", "f4"),
        ("power", "f4"),
        ("illuminance", "f4"),
        ("filename", "U100"),
        ("coneangle", "f4"),
        ("conedeltaangle", "f4"),
        ("twosided", "bool"),
    ])

    lights_np = np.zeros(len(lights), dtype=dtype)
    for j, light in enumerate(lights):
        lights_np[j] = (
            light["type"],
            light["is_area"],
            tuple(light["from"]),
            tuple(light["to"]),
            tuple(light["I"]),
            tuple(light["L"]),
            light["scale"],
            light["power"],
            light["illuminance"],
            light["filename"],
            light["coneangle"],
            light["conedeltaangle"],
            light["twosided"],
        )

    return lights_np, light_counts

def print_light_data(lights_np):
    """ Prints light data with property labels for clarity. """
    for i, light in enumerate(lights_np):
        print(f"\nLight {i + 1}:")
        print(f"   Type:          {light['type']}")
        print(f"   Area Light:    {light['is_area']}")
        print(f"   From:          {light['from']}")
        print(f"   To:            {light['to']}")
        print(f"   Intensity I:   {light['I']}")
        print(f"   Radiance L:    {light['L']}")
        print(f"   Scale:         {light['scale']}")
        print(f"   Power:         {light['power']}")
        print(f"   Illuminance:   {light['illuminance']}")
        print(f"   Filename:      {light['filename']}")
        print(f"   Cone Angle:    {light['coneangle']}°")
        print(f"   Delta Angle:   {light['conedeltaangle']}°")
        print(f"   Two-Sided:     {light['twosided']}")
        print("-" * 50)

# ----------------------------------------------
# Example Usage
# ----------------------------------------------

lights_np, light_counts = parse_pbrt_lights_and_counts(file_path)

print("Light Counts:", light_counts)
print("\nLight Types in Scene:")
print_light_data(lights_np)


Light Counts: {'total_lights': 1, 'point': 0, 'spot': 0, 'distant': 0, 'infinite': 0, 'goniometric': 0, 'projection': 0, 'area': 1}

Light Types in Scene:

Light 1:
   Type:          diffuse
   Area Light:    True
   From:          [0. 0. 0.]
   To:            [0. 0. 1.]
   Intensity I:   [1. 1. 1.]
   Radiance L:    [1000. 1000. 1000.]
   Scale:         1.0
   Power:         0.0
   Illuminance:   0.0
   Filename:      
   Cone Angle:    30.0°
   Delta Angle:   5.0°
   Two-Sided:     False
--------------------------------------------------


In [132]:
import numpy as np
import re

# --- Helper Functions ---

def clean_brackets(s):
    """
    Remove leading and trailing square brackets and quotes from a string.
    E.g. '[ 0.63 0.065 0.05 ]' becomes '0.63 0.065 0.05'
    """
    return s.strip().strip("[]").strip('"')

def parse_value(param_type, raw_value):
    """
    Convert a PBRT parameter string (e.g. "[ 0.63 0.065 0.05 ]") to an appropriate Python value.
    For "texture" and "string" types, return the first token.
    """
    cleaned = clean_brackets(raw_value)
    tokens = re.split(r'\s+', cleaned.strip())
    if not tokens or tokens[0] == "":
        return None
    try:
        if param_type == "float":
            return float(tokens[0])
        elif param_type == "integer":
            return int(tokens[0])
        elif param_type in ["point3", "vector3", "rgb", "spectrum"]:
            if len(tokens) >= 3:
                return tuple(float(tok) for tok in tokens[:3])
            else:
                return None
        elif param_type == "point2":
            if len(tokens) >= 2:
                return tuple(float(tok) for tok in tokens[:2])
            else:
                return None
        elif param_type == "integer[]":
            return list(map(int, tokens))
        elif param_type == "bool":
            return tokens[0].lower() == "true"
        elif param_type in ["string", "texture"]:
            return tokens[0]
        else:
            return cleaned
    except Exception:
        return None

def ensure_tuple3(val):
    """
    Convert a value (or return NaNs) into a 3-element tuple of floats.
    This is used for constant numeric parameters (like reflectance) that are expected to have three numbers.
    """
    if val is None:
        return (np.nan, np.nan, np.nan)
    if isinstance(val, (tuple, list)):
        try:
            if len(val) >= 3:
                return tuple(float(x) for x in val[:3])
        except Exception:
            return (np.nan, np.nan, np.nan)
    if isinstance(val, str):
        tokens = val.strip().split()
        if len(tokens) >= 3:
            try:
                return tuple(float(tok) for tok in tokens[:3])
            except Exception:
                return (np.nan, np.nan, np.nan)
    try:
        f = float(val)
        return (f, f, f)
    except Exception:
        return (np.nan, np.nan, np.nan)

def to_tuple3(val):
    """
    Convert the input value into a 3-element tuple of floats.
    If the value contains three numbers (as a tuple, list, or in a string), convert them.
    Otherwise, if it can be interpreted as a single float, promote it to a 3-tuple.
    """
    if val is None:
        return (np.nan, np.nan, np.nan)
    if isinstance(val, (tuple, list)):
        try:
            if len(val) >= 3:
                return tuple(float(x) for x in val[:3])
        except Exception:
            return (np.nan, np.nan, np.nan)
    if isinstance(val, str):
        tokens = val.strip().split()
        if len(tokens) >= 3:
            try:
                return tuple(float(tok) for tok in tokens[:3])
            except Exception:
                return (np.nan, np.nan, np.nan)
        else:
            try:
                f = float(val)
                return (f, f, f)
            except Exception:
                return (np.nan, np.nan, np.nan)
    try:
        f = float(val)
        return (f, f, f)
    except Exception:
        return (np.nan, np.nan, np.nan)

# --- PBRT Material & Texture Parsing ---

def parse_pbrt_materials(filename):
    """
    Parses a PBRT file to extract both material definitions and texture definitions.

    It handles:
      - "MakeNamedMaterial" and "Material" directives to define materials.
      - "NamedMaterial" to reference an already defined material.
      - "Texture" for texture definitions.
      - "MediumInterface" and AttributeBegin/End for medium scoping.

    For material parameters of type "texture" (e.g., "texture reflectance"),
    the texture name is stored in a key "tex_reflectance". Later, if the texture was
    defined in the file, its information (such as its filename) is extracted and stored
    in a new key "tex_filename" (which is then later printed).

    Returns a NumPy structured array with one record per unique material.
    """
    # Dictionaries to hold materials and textures (keyed by name)
    materials = {}
    textures = {}
    current_medium = ""
    medium_stack = []

    with open(filename, "r") as f:
        lines = f.readlines()

    i = 0
    while i < len(lines):
        line = lines[i].rstrip()
        if not line.strip():
            i += 1
            continue

        stripped = line.strip()
        tokens = stripped.split()
        if not tokens:
            i += 1
            continue

        # Process attribute scoping.
        if tokens[0] == "AttributeBegin":
            medium_stack.append(current_medium)
            i += 1
            continue
        if tokens[0] == "AttributeEnd":
            if medium_stack:
                current_medium = medium_stack.pop()
            i += 1
            continue

        # Process MediumInterface directives.
        if tokens[0] == "MediumInterface":
            args = [t.strip('"') for t in tokens[1:]]
            if len(args) == 1:
                current_medium = args[0]
            elif len(args) >= 2:
                current_medium = args[0] if args[0] != "" else args[1]
            i += 1
            continue

        # --- Handle Texture Definitions ---
        if tokens[0] == "Texture":
            # Expected form:
            # Texture "TextureName" "spectrum" "imagemap"
            tex_name = tokens[1].strip('"') if len(tokens) > 1 else ""
            tex_data_type = tokens[2].strip('"') if len(tokens) > 2 else ""
            tex_type = tokens[3].strip('"') if len(tokens) > 3 else ""
            texture = {
                "name": tex_name,
                "data_type": tex_data_type,
                "tex_type": tex_type,
                "filter": None,
                "filename": None
            }
            i += 1
            # Process parameter lines that belong to the texture definition.
            while i < len(lines) and lines[i] and lines[i][0].isspace():
                param_line = lines[i].strip()
                parts = param_line.split(maxsplit=2)
                if len(parts) < 3:
                    i += 1
                    continue
                # IMPORTANT: Strip quotes from the type token.
                param_type = parts[0].strip('"')
                param_key = parts[1].strip('"')
                raw_value = parts[2]
                value = parse_value(param_type, raw_value)
                if value is not None:
                    texture[param_key] = value
                i += 1
            textures[tex_name] = texture
            continue

        # --- Handle Material Definitions ---
        if tokens[0] in ["Material", "MakeNamedMaterial"]:
            # The second token is the material name.
            mat_name = tokens[1].strip('"')
            material = {
                "name": mat_name,
                "type": None,
                "reflectance": None,      # constant reflectance (if provided)
                "transmittance": None,
                "eta": None,
                "k": None,
                "roughness": None,
                "uroughness": None,
                "vroughness": None,
                "remaproughness": None,
                "displacement": None,
                "normalmap": None,
                "filename": None,
                "scale": None,
                "medium": current_medium,
                "tex_reflectance": "",   # will store the texture reference name
                "tex_filename": ""       # will be filled in if the texture is defined
            }
            i += 1
            # Process parameter lines (any line that begins with whitespace)
            while i < len(lines) and lines[i] and lines[i][0].isspace():
                param_line = lines[i].strip()
                parts = param_line.split(maxsplit=2)
                if len(parts) < 3:
                    i += 1
                    continue
                # IMPORTANT: Strip quotes from the type token!
                param_type = parts[0].strip('"')
                param_key = parts[1].strip('"')
                raw_value = parts[2]
                value = parse_value(param_type, raw_value)
                if value is not None:
                    if param_type == "texture":
                        # Store texture references in a separate key.
                        material["tex_" + param_key] = value
                    else:
                        material[param_key] = value
                i += 1
            materials[mat_name] = material
            continue

        # Process NamedMaterial activation.
        if tokens[0] == "NamedMaterial":
            mat_name = tokens[1].strip('"')
            if mat_name in materials:
                # Update medium if needed.
                if current_medium != "":
                    materials[mat_name]["medium"] = current_medium
            else:
                # Create a stub if not defined.
                materials[mat_name] = {
                    "name": mat_name,
                    "type": None,
                    "reflectance": None,
                    "transmittance": None,
                    "eta": None,
                    "k": None,
                    "roughness": None,
                    "uroughness": None,
                    "vroughness": None,
                    "remaproughness": None,
                    "displacement": None,
                    "normalmap": None,
                    "filename": None,
                    "scale": None,
                    "medium": current_medium,
                    "tex_reflectance": "",
                    "tex_filename": ""
                }
            i += 1
            continue

        i += 1

    # After processing all lines, for any material that has a texture reference, look it up.
    for mat in materials.values():
        tex_ref = mat.get("tex_reflectance", "")
        if tex_ref and tex_ref in textures:
            tex_def = textures[tex_ref]
            # For example, store the texture's filename in a new key.
            mat["tex_filename"] = tex_def.get("filename", "")

    # Define a NumPy structured dtype. Note we add a "tex_filename" field.
    dtype = np.dtype([
        ("name", "U64"),
        ("type", "U32"),
        ("reflectance", "3f4"),
        ("tex_reflectance", "U128"),
        ("tex_filename", "U128"),
        ("transmittance", "3f4"),
        ("eta", "3f4"),
        ("k", "3f4"),
        ("roughness", "f4"),
        ("uroughness", "f4"),
        ("vroughness", "f4"),
        ("remaproughness", "bool"),
        ("displacement", "f4"),
        ("normalmap", "U128"),
        ("filename", "U128"),
        ("scale", "f4"),
        ("medium", "U64"),
    ])

    # Convert our materials dictionary to a list.
    material_list = list(materials.values())
    materials_np = np.zeros(len(material_list), dtype=dtype)
    for j, mat in enumerate(material_list):
        # For reflectance, if a texture was provided, leave constant reflectance as NaNs.
        tex_refl = mat.get("tex_reflectance", "")
        if tex_refl:
            refl_val = (np.nan, np.nan, np.nan)
        else:
            refl_val = ensure_tuple3(mat.get("reflectance"))
        eta_conv = to_tuple3(mat.get("eta"))
        k_conv = to_tuple3(mat.get("k"))
        materials_np[j] = (
            mat.get("name", ""),
            mat.get("type", ""),
            refl_val,
            tex_refl,
            mat.get("tex_filename", ""),
            ensure_tuple3(mat.get("transmittance")),
            eta_conv,
            k_conv,
            float(mat.get("roughness")) if mat.get("roughness") is not None else np.nan,
            float(mat.get("uroughness")) if mat.get("uroughness") is not None else np.nan,
            float(mat.get("vroughness")) if mat.get("vroughness") is not None else np.nan,
            bool(mat.get("remaproughness")) if mat.get("remaproughness") is not None else False,
            float(mat.get("displacement")) if mat.get("displacement") is not None else np.nan,
            mat.get("normalmap", "") if mat.get("normalmap") is not None else "",
            mat.get("filename", "") if mat.get("filename") is not None else "",
            float(mat.get("scale")) if mat.get("scale") is not None else np.nan,
            mat.get("medium", "")
        )
    return materials_np

# --- Example Usage ---

materials_np = parse_pbrt_materials(file_path)
for idx, mat in enumerate(materials_np):
    print(f"Material {idx+1}:")
    print(f"  Name:            {mat['name']}")
    print(f"  Type:            {mat['type']}")
    # If a texture reference was provided for reflectance, print that.
    if mat['tex_reflectance']:
        print(f"  Texture Reflectance: {mat['tex_reflectance']}")
        if mat['tex_filename']:
            print(f"  Texture Filename:    {mat['tex_filename']}")
    else:
        if not np.isnan(mat['reflectance'][0]):
            print(f"  Reflectance:     {mat['reflectance']}")
    if not np.isnan(mat['transmittance'][0]):
        print(f"  Transmittance:   {mat['transmittance']}")
    if not np.isnan(mat['eta'][0]):
        print(f"  Eta:             {mat['eta']}")
    if not np.isnan(mat['k'][0]):
        print(f"  k:               {mat['k']}")
    if not np.isnan(mat['roughness']):
        print(f"  Roughness:       {mat['roughness']}")
    if not np.isnan(mat['uroughness']):
        print(f"  URoughness:      {mat['uroughness']}")
    if not np.isnan(mat['vroughness']):
        print(f"  VRoughness:      {mat['vroughness']}")
    print(f"  RemapRoughness:  {mat['remaproughness']}")
    if not np.isnan(mat['displacement']):
        print(f"  Displacement:    {mat['displacement']}")
    if mat['normalmap']:
        print(f"  NormalMap:       {mat['normalmap']}")
    if mat['filename']:
        print(f"  Filename:        {mat['filename']}")
    if not np.isnan(mat['scale']):
        print(f"  Scale:           {mat['scale']}")
    if mat['medium']:
        print(f"  Medium:          {mat['medium']}")
    print("-" * 50)


Material 1:
  Name:            Landscape
  Type:            "diffuse"
  Texture Reflectance: "Texture01"
  RemapRoughness:  False
--------------------------------------------------
Material 2:
  Name:            Table
  Type:            "diffuse"
  Texture Reflectance: "Texture02"
  RemapRoughness:  False
--------------------------------------------------
Material 3:
  Name:            DoorHandle
  Type:            "conductor"
  Eta:             [1.65746  0.880369 0.521229]
  k:               [9.223869 6.269523 4.837001]
  URoughness:      0.25
  VRoughness:      0.25
  RemapRoughness:  False
--------------------------------------------------
Material 4:
  Name:            Door
  Type:            "diffuse"
  Texture Reflectance: "Texture03"
  RemapRoughness:  False
--------------------------------------------------
Material 5:
  Name:            Diffuse
  Type:            "diffuse"
  Reflectance:     [0.8 0.8 0.8]
  RemapRoughness:  False
-----------------------------------------------

In [133]:
import numpy as np
import re

# ---------------------------------------------------------------------------
# Helper Functions (for parameter parsing)
# ---------------------------------------------------------------------------
def parse_value(param_type, raw_value):
    """
    Convert a PBRT parameter string (e.g. "[ 0.63 0.065 0.05 ]") into a Python value.
    For "string" and "texture" types, the first token is returned.
    """
    cleaned = raw_value.strip().strip("[]").strip('"')
    tokens = re.split(r'\s+', cleaned.strip())
    if not tokens or tokens[0] == "":
        return None
    try:
        if param_type == "float":
            # Even if multiple tokens are present, for float we take the first one.
            return float(tokens[0])
        elif param_type == "integer":
            # If there is more than one token, return an array of ints.
            if len(tokens) > 1:
                return np.array([int(tok) for tok in tokens])
            else:
                return int(tokens[0])
        elif param_type in ["point3", "point2", "vector", "vector3", "rgb", "spectrum"]:
            # Return as a NumPy array of floats.
            return np.array([float(tok) for tok in tokens])
        elif param_type == "point":
            # For an arbitrary-length point.
            return np.array([float(tok) for tok in tokens])
        elif param_type == "bool":
            return tokens[0].lower() == "true"
        elif param_type in ["string", "texture"]:
            return tokens[0]
        else:
            return cleaned
    except Exception:
        return None

def accumulate_property_line(lines, start_index):
    """
    Accumulates successive lines (starting at index start_index) that belong to the
    same property definition until a closing bracket ("]") is found.

    Returns:
      - The accumulated line (as a single string).
      - The index of the line after the property.
    """
    acc_line = lines[start_index].rstrip()
    i = start_index + 1
    # Keep adding lines until we find a ']' in the accumulated text.
    while ']' not in acc_line and i < len(lines):
        acc_line += " " + lines[i].strip()
        i += 1
    return acc_line, i

def parse_property_line(line):
    """
    Given a line (or an accumulated multi-line string) that is expected to start with a
    quoted parameter declaration (e.g. "float radius") followed either by a bracketed value
    or by another quoted string, extract and return a tuple (ptype, key, value).

    If a bracketed value is present, it is used; otherwise, if another quoted token is found,
    that token is used as the value.

    If parsing fails, returns (None, None, None).
    """
    # Extract the first quoted string (the declaration).
    m_decl = re.search(r'"([^"]+)"', line)
    if not m_decl:
        return None, None, None
    declaration = m_decl.group(1)  # e.g. "float filter" or "float radius"
    parts = declaration.split()
    if len(parts) < 2:
        return None, None, None
    ptype, key = parts[0], parts[1]

    # Try to extract a bracketed value (using DOTALL to capture newlines).
    m_val = re.search(r'\[([\s\S]+?)\]', line)
    if m_val:
        raw_value = f"[{m_val.group(1).strip()}]"
        value = parse_value(ptype, raw_value)
        return ptype, key, value
    else:
        # If no bracket is found, look for a second quoted string.
        all_quotes = re.findall(r'"([^"]+)"', line)
        if len(all_quotes) >= 2:
            value = all_quotes[1]
            return ptype, key, value
    return ptype, key, None

def parse_transform_line(line):
    """
    Given a line starting with "Transform", extract the numbers inside the brackets.
    If 16 numbers are found, returns a NumPy array reshaped as (4,4). Otherwise, returns None.
    """
    m = re.search(r'\[([^\]]+)\]', line)
    if not m:
        return None
    numbers = m.group(1).split()
    if len(numbers) < 16:
        return None
    try:
        mat = np.array([float(n) for n in numbers]).reshape((4,4))
        return mat
    except Exception:
        return None

# ---------------------------------------------------------------------------
# Unified PBRT Parser for Textures and Shapes with State Tracking
# ---------------------------------------------------------------------------
def parse_pbrt_file(filename):
    """
    Parses a PBRT file to extract textures, shapes, and to track state directives
    (Transform, NamedMaterial, MediumInterface). Each shape definition is augmented
    with the current transformation matrix, active material, and medium. Texture
    definitions are stored in a separate dictionary.

    Returns:
      - shapes: a list of shape dictionaries.
      - shape_counts: a dictionary counting the number of shapes per type.
      - textures: a dictionary mapping texture names to texture parameter dictionaries.
    """
    shapes = []
    shape_counts = {}
    textures = {}

    # State variables.
    current_transform = None
    current_material = None
    current_medium = None

    with open(filename, "r") as f:
        lines = f.readlines()

    i = 0
    while i < len(lines):
        line = lines[i]
        stripped = line.strip()
        if not stripped:
            i += 1
            continue

        # Process state directives.
        if stripped.startswith("Transform"):
            current_transform = parse_transform_line(stripped)
            i += 1
            continue

        if stripped.startswith("NamedMaterial"):
            tokens = stripped.split()
            if len(tokens) >= 2:
                current_material = tokens[1].strip('"')
            i += 1
            continue

        if stripped.startswith("MediumInterface"):
            tokens = stripped.split()
            if len(tokens) >= 2:
                medium_candidate = tokens[1].strip('"')
                if medium_candidate == "" and len(tokens) >= 3:
                    current_medium = tokens[2].strip('"')
                else:
                    current_medium = medium_candidate
            i += 1
            continue

        # Process Texture directives.
        if stripped.startswith("Texture"):
            tokens = stripped.split()
            if len(tokens) < 2:
                i += 1
                continue
            tex_name = tokens[1].strip('"')
            tex_data = {"name": tex_name}
            i += 1
            while i < len(lines) and lines[i].lstrip().startswith('"'):
                acc_line, i = accumulate_property_line(lines, i)
                ptype, key, value = parse_property_line(acc_line)
                if key is not None:
                    tex_data[key] = value
            textures[tex_name] = tex_data
            continue

        # Process Shape directives.
        if stripped.startswith("Shape"):
            tokens = stripped.split()
            if len(tokens) < 2:
                i += 1
                continue
            shape_type = tokens[1].strip('"')
            shape_counts[shape_type] = shape_counts.get(shape_type, 0) + 1

            shape_data = {"shape": shape_type,
                          "transform": current_transform,
                          "material": current_material,
                          "medium": current_medium}
            i += 1
            while i < len(lines) and lines[i].lstrip().startswith('"'):
                acc_line, i = accumulate_property_line(lines, i)
                ptype, key, value = parse_property_line(acc_line)
                if key is not None:
                    shape_data[key] = value
            shapes.append(shape_data)
            continue

        i += 1

    return shapes, shape_counts, textures

# ---------------------------------------------------------------------------
# Conversion to a NumPy Structured Array (for shapes)
# ---------------------------------------------------------------------------
def shapes_to_numpy(shapes_list):
    """
    Converts a list of shape dictionaries into a NumPy structured array.
    Some fields (like 'transform') are stored as objects.
    """
    dtype = np.dtype([
        ("shape", "U20"),
        ("transform", "O"),   # e.g., a 4x4 NumPy array or None
        ("material", "U50"),
        ("medium", "U50"),
        ("P", "O"),
        ("indices", "O"),
        ("radius", "f4"),
        ("N", "O"),
        ("uv", "O"),
        ("extra", "O"),
    ])
    out = np.empty(len(shapes_list), dtype=dtype)
    for j, sh in enumerate(shapes_list):
        out[j] = (
            sh.get("shape", ""),
            sh.get("transform", None),
            sh.get("material", ""),
            sh.get("medium", ""),
            sh.get("P", None),
            sh.get("indices", None),
            sh.get("radius", np.nan),
            sh.get("N", None),
            sh.get("uv", None),
            sh.get("extra", {})
        )
    return out

# ---------------------------------------------------------------------------
# Example Usage
# ---------------------------------------------------------------------------

shapes_list, shape_counts, textures = parse_pbrt_file(file_path)

print("Shape Counts:", shape_counts)
print("\nTextures Extracted:")
for name, tex in textures.items():
    print(f"   Texture '{name}': {tex}")
print("\nShapes Extracted:")
for idx, shape in enumerate(shapes_list):
    print(f"\nShape {idx + 1}:")
    print(f"   Shape Type: {shape.get('shape', 'N/A')}")
    print(f"   Transform: {shape.get('transform', None)}")
    print(f"   Material: {shape.get('material', '')}")
    print(f"   Medium: {shape.get('medium', '')}")
    for key, value in shape.items():
        if key in ["shape", "transform", "material", "medium"]:
            continue
        print(f"   {key}: {value}")

shapes_np = shapes_to_numpy(shapes_list)
print("\nStructured NumPy Array of Shapes:")
print(shapes_np)


Shape Counts: {'trianglemesh': 1, 'plymesh': 21}

Textures Extracted:
   Texture 'Texture01': {'name': 'Texture01', 'filter': 'textures/landscape-with-a-lake.tga'}
   Texture 'Texture02': {'name': 'Texture02', 'filter': 'textures/Good'}
   Texture 'Texture03': {'name': 'Texture03', 'filter': 'textures/cherry-wood-texture.tga'}

Shapes Extracted:

Shape 1:
   Shape Type: trianglemesh
   Transform: [[ 0.137285   -0.0319925  -0.990015   -0.        ]
 [-0.          0.999478   -0.0322983  -0.        ]
 [-0.990531   -0.00443405 -0.137212   -0.        ]
 [-2.84124    -1.49616     3.74927     1.        ]]
   Material: Light
   Medium: None
   uv: [0. 0. 1. 0. 1. 1. 0. 1.]
   N: 0 -1.03553e-7 -1 0 -1.03553e-7 -1 0 -1.03553e-7 -1 0 -1.03553e-7 -1
   P: [-5.16954   2.82792  -4.44377  -3.70865   2.82792  -4.44377  -3.70865
  0.185195 -4.44377  -5.16954   0.185195 -4.44377 ]
   indices: [0 1 2 0 2 3]

Shape 2:
   Shape Type: plymesh
   Transform: [[ 0.137285   -0.0319925  -0.990015   -0.        ]
 

In [119]:
shapes_list

[{'shape': 'trianglemesh',
  'transform': array([[ 1.00000e+00, -0.00000e+00,  1.50996e-07, -0.00000e+00],
         [-0.00000e+00,  1.00000e+00, -0.00000e+00, -0.00000e+00],
         [ 1.50996e-07, -0.00000e+00, -1.00000e+00, -0.00000e+00],
         [-1.13687e-13, -1.00000e+00,  7.00000e+00,  1.00000e+00]]),
  'material': 'Floor',
  'medium': None,
  'uv': None},
 {'shape': 'trianglemesh',
  'transform': array([[ 1.00000e+00, -0.00000e+00,  1.50996e-07, -0.00000e+00],
         [-0.00000e+00,  1.00000e+00, -0.00000e+00, -0.00000e+00],
         [ 1.50996e-07, -0.00000e+00, -1.00000e+00, -0.00000e+00],
         [-1.13687e-13, -1.00000e+00,  7.00000e+00,  1.00000e+00]]),
  'material': 'Ceiling',
  'medium': None,
  'uv': None},
 {'shape': 'trianglemesh',
  'transform': array([[ 1.00000e+00, -0.00000e+00,  1.50996e-07, -0.00000e+00],
         [-0.00000e+00,  1.00000e+00, -0.00000e+00, -0.00000e+00],
         [ 1.50996e-07, -0.00000e+00, -1.00000e+00, -0.00000e+00],
         [-1.13687e-13, -

In [ ]:
from utils.pbrt_parser import parse_pbrt

parse_pbrt("cornell-box.pbrt", num_triangles, num_spheres, num_lights, num_primitives,
           triangles, spheres, lights, primitives, materials, light_sampler)
print("Parsing completed!")


In [ ]:
hffoo

In [ ]:
from utils.mesh import parse_scene, extract_material_data_from_mesh
import pyvista as pv

pv.set_jupyter_backend('trame')


vertices, faces, material_data = parse_scene(obj_path)

# Print material data
print("Material Data:")
print(material_data)

# Print vertices
print("\nVertices:")
print(vertices)

# Print faces
print("\nFaces Array:")
print(faces)

# Now you can load vertices and faces_array into PyVista
# Example:
mesh = pv.PolyData(vertices, faces)


# Attach all material properties to the mesh
mesh.cell_data['face_idx'] = material_data['face_idx']
mesh.cell_data['diffuse'] = material_data['diffuse']
mesh.cell_data['ambient'] = material_data['ambient']
mesh.cell_data['specular'] = material_data['specular']
mesh.cell_data['emission'] = material_data['emission']
mesh.cell_data['shininess'] = material_data['shininess']
mesh.cell_data['ior'] = material_data['ior']
mesh.cell_data['opacity'] = material_data['opacity']
mesh.cell_data['illum'] = material_data['illum']
mesh.cell_data['is_light'] = material_data['is_light']

# Triangulate the mesh
triangulated_mesh = mesh.triangulate()

# Extract material data from the triangulated mesh
material_data = extract_material_data_from_mesh(triangulated_mesh)

# Print extracted material data to verify
print(material_data)



# Create the left mirror sphere
left_sphere_center = (-0.31, 0.495, 0.175)
left_sphere_radius = 0.495
left_sphere = pv.Sphere(radius=left_sphere_radius, center=left_sphere_center)

# Create the right transparent glass sphere
right_sphere_center = (0.49, 0.245, -0.225)
right_sphere_radius = 0.245
right_sphere = pv.Sphere(radius=right_sphere_radius, center=right_sphere_center)



# Visualize the triangulated mesh to check if colors are mapped properly
plotter = pv.Plotter()


# Set the initial camera position and focal point
plotter.camera_position = 'xy'  # Align the camera view


plotter.add_mesh(triangulated_mesh, scalars='diffuse', rgb=True, show_edges=True)


# Add the spheres to the plotter
plotter.add_mesh(left_sphere, color='silver', specular=1.0, smooth_shading=True)  # Mirror-like sphere
plotter.add_mesh(right_sphere, color='white', opacity=0.5, smooth_shading=True)  # Transparent sphere


# Reset the camera to automatically fit the scene
plotter.reset_camera()  # This does what "Reset camera" in the GUI does

# Show the plotter window
plotter.show()

# Extract the updated camera parameters after resetting the camera
camera_position = plotter.camera_position[0]  # Camera position
camera_focal_point = plotter.camera_position[1]  # Look-at point
camera_up_vector = plotter.camera.up  # Up direction

# Convert these to numpy arrays or Taichi vectors as needed
_position = np.array(camera_position)
_look_at = np.array(camera_focal_point)
_up = np.array(camera_up_vector)


In [ ]:
_position, _look_at, _up

In [ ]:
print(triangulated_mesh.center)

In [ ]:
# Example: Check how many faces have illum=7
illum_1 = material_data[material_data['illum'] == 1].shape[0]
illum_2 = material_data[material_data['illum'] == 2].shape[0]
illum_5 = material_data[material_data['illum'] == 5].shape[0]
illum_7 = material_data[material_data['illum'] == 7].shape[0]

# Number of faces with illum=7
total_faces = material_data.shape[0]

# Print the result
print(total_faces==(illum_1+illum_2+illum_5+illum_7))
print(total_faces, illum_1, illum_2, illum_5, illum_7)

In [ ]:
# Extract vertices
vertices = triangulated_mesh.points.astype(np.float32)
n_vertices = vertices.shape[0]

# Extract faces directly after triangulation
faces = triangulated_mesh.faces.reshape(-1, 4)[:, 1:4]  # Reshape and remove the first column (number of points per face)
faces = faces.astype(np.int32)
n_faces = faces.shape[0]

n_triangles = n_faces

n_lights = illum_1  # assumed emissive surfaces have illum=2

print(n_triangles, n_faces, n_vertices)

In [ ]:
# Create Taichi fields
vertices_field = ti.Vector.field(3, dtype=ti.f32, shape=(n_vertices,))
faces_field = ti.Vector.field(3, dtype=ti.i32, shape=(n_faces,))

vertices_field.from_numpy(vertices)
faces_field.from_numpy(faces)

In [ ]:
# from primitives.triangle import Triangle
#
# # Create a Taichi field for the triangles
# TS = ti.root.dense(ti.i, n_triangles)
# triangles = Triangle.field()
# TS.place(triangles)

In [ ]:
n_spheres = 2 # will describe later
n_primitives = n_triangles + n_spheres

In [ ]:
from primitives.primitives import Sphere

if n_spheres==0:
    sphere_field_size = 1
else:
    sphere_field_size = n_spheres

SS = ti.root.dense(ti.i, sphere_field_size)
spheres = Sphere.field()  # shape=(n_spheres)
SS.place(spheres)

In [ ]:
from base.materials import Material

# Create a Taichi field for the materials

MS = ti.root.dense(ti.i, n_triangles)
materials = Material.field()
MS.place(materials)

In [ ]:
from primitives.primitives import Primitive

# Create a Taichi field for the materials
PS = ti.root.dense(ti.i, n_primitives)
primitives = Primitive.field()
PS.place(primitives)

In [ ]:
from base.lights import DiffuseAreaLight

# Create a Taichi field for light sources
LS = ti.root.dense(ti.i, n_lights)
lights = DiffuseAreaLight.field()
LS.place(lights)

In [ ]:
# Extract the 'face_idx' field
face_idx = material_data['face_idx']

# Check if all 'face_idx' values are unique
are_unique = len(face_idx) == len(np.unique(face_idx))

print("All face_idx are unique:", are_unique)

In [ ]:
len(material_data) == n_primitives

In [ ]:
materials.from_numpy(material_data)

In [ ]:
materials[11]['face_idx'] == 11

In [ ]:
lights.shape

In [ ]:
from utils.misc import max_component
from base.bsdf import BXDF_DIFFUSE_REFLECTION, BXDF_SPECULAR_REFLECTION, BXDF_DIFFUSE_TRANSMISSION, BXDF_SPECULAR_TRANSMISSION

@ti.kernel
def populate_shapes():
    light_idx = 0
    for i in range(primitives.shape[0]-n_spheres):
        # Set up the geometry for the primitive (triangle in this case)
        primitives[i].triangle.vertex_1 = vertices_field[faces_field[i][0]]
        primitives[i].triangle.vertex_2 = vertices_field[faces_field[i][1]]
        primitives[i].triangle.vertex_3 = vertices_field[faces_field[i][2]]
        primitives[i].triangle.centroid = (primitives[i].triangle.vertex_1 + primitives[i].triangle.vertex_2 + primitives[i].triangle.vertex_3) / 3.0
        primitives[i].triangle.edge_1 = primitives[i].triangle.vertex_2 - primitives[i].triangle.vertex_1
        primitives[i].triangle.edge_2 = primitives[i].triangle.vertex_3 - primitives[i].triangle.vertex_1
        primitives[i].triangle.normal = normalize(cross(primitives[i].triangle.edge_1, primitives[i].triangle.edge_2))

        # Initialize the BSDF frame
        # primitives[i].bsdf.init_frame(primitives[i].triangle.normal)

        primitives[i].bsdf.diffuse.type = 0
        primitives[i].bsdf.transmit.type = 1
        primitives[i].bsdf.dielectric.type = 2
        primitives[i].bsdf.conductor.type = 3
        primitives[i].bsdf.mirror.type = 4
        primitives[i].bsdf.specular.type = 5

        # Determine the BxDF type based on material properties
        illum = materials[i].illum

        # if illum == 0 or illum == 1:  # Diffuse materials
        #     primitives[i].bsdf.add_diffuse(materials[i].diffuse)
        # elif illum == 2:  # Diffuse + Specular (using Conductor BxDF for specular)
        #     roughness = 1.0 - (materials[i].shininess / 1000.0)
        #     primitives[i].bsdf.add_conductor(materials[i].diffuse, vec3(0.0), roughness)
        # elif illum == 3:  # Diffuse + Specular + Reflection (using Conductor BxDF)
        #     roughness = 1.0 - (materials[i].shininess / 1000.0)
        #     primitives[i].bsdf.add_conductor(materials[i].diffuse, vec3(0.0), roughness)
        # elif illum == 4 or illum == 6:  # Transparent materials (Dielectric BxDF)
        #     primitives[i].bsdf.add_dielectric(materials[i].ior, 0.0)  # Assume perfectly smooth dielectric
        # elif illum == 5 or illum == 7:  # Mirror reflection (using Conductor BxDF)
        #     primitives[i].bsdf.add_conductor(vec3(1.5), vec3(0.0), 0.0)  # Perfectly smooth conductor
        # else:  # Default to Diffuse
        #     primitives[i].bsdf.add_diffuse(materials[i].diffuse)

        if illum == 1 or illum == 2:  # Diffuse materials
            primitives[i].bsdf.add_diffuse(materials[i].diffuse)
        if illum == 5:  # Mirror materials
            primitives[i].bsdf.add_mirror(materials[i].specular)
        elif illum == 6:  # Metals (using Conductor BxDF)
            eta_silver = vec3(0.155, 0.116, 0.138)
            k_silver = vec3(4.828, 3.122, 2.146)
            roughness = 0.0
            primitives[i].bsdf.add_conductor(eta_silver, k_silver, roughness)
        elif illum == 7:  # Transparent materials (Dielectric BxDF)
            primitives[i].bsdf.add_dielectric(materials[i].ior, materials[i].specular, 0.1)  # Assume perfectly smooth dielectric
        else:  # Default to Diffuse
            primitives[i].bsdf.add_diffuse(materials[i].diffuse)


        # Handle emissive materials (lights)
        if max_component(materials[i].emission) > 0:
            primitives[i].is_light = 1
            primitives[i].light_idx = light_idx
            lights[light_idx].shape_idx = i  # Link the light to the primitive
            lights[light_idx].two_sided = 0  # Assuming lights are one-sided
            lights[light_idx].Le = materials[i].emission
            light_idx += 1
        else:
            primitives[i].is_light = 0

        # Assign material to the primitive
        primitives[i].material = materials[i]

        # Set the shape type, assuming 0 is for triangle
        primitives[i].shape_type = 0

        if materials[i].face_idx == i:
            primitives[i].material = materials[i]
        else:
            print("face mismatch!!")

    print("Shapes populated:", primitives.shape[0])


# Call the kernel to populate the triangles
populate_shapes()

In [ ]:
@ti.kernel
def add_spheres():

    # Define the BSDF types for easy reference
    primitives[n_triangles].bsdf.diffuse.type = 0
    primitives[n_triangles].bsdf.transmit.type = 1
    primitives[n_triangles].bsdf.dielectric.type = 2
    primitives[n_triangles].bsdf.conductor.type = 3
    primitives[n_triangles].bsdf.mirror.type = 4
    primitives[n_triangles].bsdf.specular.type = 5

    # Add the left mirror sphere (Positioned on the left side of the box)
    primitives[n_triangles].shape_type = 1  # 1 indicates a sphere
    primitives[n_triangles].sphere.center = vec3([-0.31, 0.495, 0.175])
    primitives[n_triangles].sphere.radius = 0.495
    # primitives[n_triangles].sphere.center = vec3([-0.41, 0.395, -0.225])
    # primitives[n_triangles].sphere.radius = 0.4
    primitives[n_triangles].is_light = 0

    # Set up the material for the left mirror sphere
    primitives[n_triangles].material.diffuse = vec3([0.03, 0.03, 0.03])
    primitives[n_triangles].material.specular = vec3([0.999, 0.999, 0.999])
    primitives[n_triangles].material.emission = vec3([0.0, 0.0, 0.0])
    primitives[n_triangles].material.shininess = 1024.0
    primitives[n_triangles].material.ior = 1.0
    primitives[n_triangles].material.opacity = 1.0
    primitives[n_triangles].material.is_light = 0

    eta_silver = vec3(0.155, 0.116, 0.138)
    k_silver = vec3(4.828, 3.122, 2.146)
    roughness = 0.0

    # Set up the BSDF for the left mirror sphere
    primitives[n_triangles].bsdf.add_conductor(eta_silver, k_silver, roughness)



    # Define the BSDF types for easy reference
    primitives[n_triangles + 1].bsdf.diffuse.type = 0
    primitives[n_triangles + 1].bsdf.transmit.type = 1
    primitives[n_triangles + 1].bsdf.dielectric.type = 2
    primitives[n_triangles + 1].bsdf.conductor.type = 3
    primitives[n_triangles + 1].bsdf.mirror.type = 4
    primitives[n_triangles + 1].bsdf.specular.type = 5

    # Add the right transparent glass sphere (Positioned on the right side of the box)
    primitives[n_triangles + 1].shape_type = 1  # 1 indicates a sphere
    primitives[n_triangles + 1].sphere.center = vec3([0.49, 0.245, -0.225])
    primitives[n_triangles + 1].sphere.radius = 0.245
    # primitives[n_triangles + 1].sphere.center = vec3([0.49, 0.395, 0.175])
    # primitives[n_triangles + 1].sphere.radius = 0.4
    primitives[n_triangles + 1].is_light = 0  # Not a light source

    # Set up the material for the right transparent glass sphere
    primitives[n_triangles + 1].material.diffuse = vec3([0.01, 0.01, 0.01])
    primitives[n_triangles + 1].material.specular = vec3([0.999, 0.999, 0.999])
    primitives[n_triangles + 1].material.emission = vec3([0.0, 0.0, 0.0])
    primitives[n_triangles + 1].material.shininess = 1024.0
    primitives[n_triangles + 1].material.ior = 1.5
    primitives[n_triangles + 1].material.opacity = 0.1
    primitives[n_triangles + 1].material.is_light = 0

    # Set up the BSDF for the right transparent glass sphere
    primitives[n_triangles + 1].bsdf.add_dielectric(primitives[n_triangles + 1].material.ior, primitives[n_triangles + 1].material.specular, 0.0)
    # primitives[n_triangles + 1].bsdf.add_specular(primitives[n_triangles + 1].material.specular, primitives[n_triangles + 1].material.ior)

# Assuming `primitives` and `n_triangles` are defined globally
add_spheres()


In [ ]:
primitives[n_triangles].bsdf.type

In [ ]:
lights[0].shape_idx

In [ ]:
# p_min = np.min(vertices)
# p_max = np.max(vertices)
#
# # centroid = vec3(bbox_center)
# centroid = mesh.center
#
# min_p = ti.field(dtype=ti.f32, shape=())
# min_p[None] = p_min
#
# centroid, min_p[None]

In [ ]:
from primitives.aabb import AABB, BVHPrimitive

# Enclose all the primitives in their individual axis-aligned bounding boxes (AABB)
BS = ti.root.dense(ti.i, n_primitives)
bvh_primitives = BVHPrimitive.field()
BS.place(bvh_primitives)

@ti.kernel
def init_bounded_boxes():
    for i in ti.ndrange(n_primitives):
        bvh_primitives[i].prim = primitives[i]
        bvh_primitives[i].prim_num = i
        min_p, max_p = primitives[i].get_bounds()
        centroid = (min_p+max_p)/2
        bvh_primitives[i].bounds = AABB(min_point=min_p, max_point=max_p, centroid=centroid)

init_bounded_boxes()

bvh_primitives.shape[0], n_primitives

In [ ]:
# bounded_boxes[3]

In [ ]:
OPS = ti.root.dense(ti.i, n_primitives)
ordered_prims = Primitive.field()
OPS.place(ordered_prims)

In [ ]:
from accelerators.bvh import BVHNode, BucketInfo

node_idx = 0

BVS = ti.root.dense(ti.i, 3 * n_primitives)
nodes = BVHNode.field()
BVS.place(nodes)


In [ ]:
nodes.shape[0]

In [ ]:

total_nodes = ti.field(ti.i32, shape=())
split_method = ti.field(ti.i32, shape=())
start = ti.field(ti.i32, shape=())
end = ti.field(ti.i32, shape=())
ordered_prims_idx = ti.field(ti.i32, shape=())
stack_ptr = ti.field(ti.i32, shape=())
max_prims_in_node = ti.field(ti.i32, shape=())
# ordered_prims_size = ti.field(ti.i32, shape=())

@ti.kernel
def init_stack():
    total_nodes[None] = 0
    split_method[None] = 0
    start[None] = 0
    end[None] = n_primitives
    ordered_prims_idx[None] = 0
    stack_ptr[None] = 0
    max_prims_in_node[None] = 4

init_stack()

In [ ]:
stack = ti.field(ti.i32, shape=(3*n_primitives, 4))
n_buckets = 12
buckets = BucketInfo.field(shape=(n_buckets))
costs = ti.field(dtype=ti.f32, shape=(n_buckets-1))

In [ ]:
from accelerators.hlbvh import build_hlbvh
from accelerators.bvh import build_bvh

start_t = time.time()

root = ti.field(ti.i32, shape=())

build_hlbvh(
    primitives,
    bvh_primitives,
    ordered_prims,
    n_primitives,       # number of actual primitives
    nodes,
    total_nodes,
    ordered_prims_idx,  # you already have this
    max_prims_in_node,
    root
)

end_t = time.time()

In [ ]:
print("Elapsed (with compilation) = %s" % (end_t - start_t))
print(f"Total Nodes: {total_nodes}")

In [ ]:
root[None]

In [ ]:
nodes[9].child_1

In [ ]:
from tests.test_bvh import print_bvh

print_bvh(nodes, total_nodes[None])

In [ ]:
from tests.test_bvh import test_bvh
max_nodes = n_primitives
debug_stack = ti.field(dtype=ti.i32, shape=(max_nodes,))
debug_ptr = ti.field(dtype=ti.i32, shape=(1,))
error_count = ti.field(dtype=ti.i32, shape=(1,))

test_bvh(nodes, debug_stack, debug_ptr, error_count)

In [ ]:
from accelerators.bvh import LinearBVHNode
from accelerators.bvh import flatten_bvh

# create a linear representation of the bvh tree (empty)

LBVS = ti.root.dense(ti.i, total_nodes[None])
linear_bvh = LinearBVHNode.field()
LBVS.place(linear_bvh)

stack_size = 2 * n_primitives - 1  # Maximum possible stack size
stack3 = ti.Vector.field(3, dtype=ti.i32, shape=stack_size)
stack_top_2 = ti.field(dtype=ti.i32, shape=())

start_t = time.time()
total_nodes = flatten_bvh(nodes, linear_bvh, root[None], stack3, stack_top_2)
end_t = time.time()
print(f"Total flattened nodes: {total_nodes}")

In [ ]:
linear_bvh[5].second_child_offset

In [ ]:
print("Elapsed (with compilation) = %s" % (end_t - start_t))

In [ ]:
from tests.test_linear_bvh import print_flattened_bvh

print_flattened_bvh(linear_bvh, total_nodes)

In [ ]:
from tests.test_linear_bvh import test_flattened_bvh

max_nodes = n_primitives
debug_stack = ti.field(dtype=ti.i32, shape=(max_nodes,))
debug_ptr = ti.field(dtype=ti.i32, shape=(1,))
error_count = ti.field(dtype=ti.i32, shape=(1,))

test_flattened_bvh(linear_bvh, debug_stack, debug_ptr, error_count)

In [ ]:
position = ti.Vector(_position.tolist()) # centroid + vec3([0.0, 0.0, min_p[None]*3])
look_at = ti.Vector(_look_at.tolist()) # normalize(centroid - position)
up = ti.Vector(_up.tolist()) # vec3([0.0, 1.0, 0.0])

In [ ]:
from taichi.math import radians, tan
from base.frame import Frame, frame_from_z
from base.camera import PerspectiveCamera

camera = PerspectiveCamera.field(shape=())

width = 256
height = 256

# zoomed_position = ti.Vector.field(3, dtype=ti.f32, shape=())
# fov = ti.field(ti.f32, shape=())
# aspect_ratio = ti.field(ti.f32, shape=())
# lens_radius = ti.field(ti.f32, shape=())
# focal_distance = ti.field(ti.f32, shape=())
# screen_dx = ti.field(ti.f32, shape=())
# screen_dy = ti.field(ti.f32, shape=())
# dx_camera = ti.Vector.field(3, dtype=ti.f32, shape=())
# dy_camera = ti.Vector.field(3, dtype=ti.f32, shape=())
# frame = Frame.field(shape=())

@ti.kernel
def init_cam():
    camera.width[None] = width
    camera.height[None] = height
    camera.fov[None] = 30.0
    camera.aspect_ratio[None] = width / float(height)
    camera.lens_radius[None] = 0.00  # Set to a small value like 0.01 to test depth of field
    camera.focal_distance[None] = (look_at - position).norm()

    direction = (look_at - position).normalized()
    zoom_factor = 1.5  # + in / - out
    camera.position[None] = position + direction * zoom_factor

    # Setup the camera frame (z points towards the scene)
    z = (look_at - camera.position[None]).normalized()
    camera.frame[None] = frame_from_z(z)

    theta = radians(camera.fov[None])
    half_height = tan(theta / 2)
    half_width = camera.aspect_ratio[None] * half_height

    # Compute the screen dimensions in camera space
    camera.screen_dx[None] = 2 * half_width
    camera.screen_dy[None] = 2 * half_height

    # Compute the pixel size in camera space
    camera.dx_camera[None] = ti.Vector([camera.screen_dx[None] / width, 0.0, 0.0])
    camera.dy_camera[None] = ti.Vector([0.0, camera.screen_dy[None] / height, 0.0])


init_cam()

In [ ]:
camera.frame[None]

In [ ]:
@ti.kernel
def test_camera():
    # Generate a ray
    s, t = 0.5, 0.5  # Center of the screen
    ray_origin, ray_dir, rx_origin, rx_dir, ry_origin, ry_dir = camera[None].generate_ray_differential(s, t)
    print(ray_origin, ray_dir, rx_origin, rx_dir, ry_origin, ry_dir)

test_camera()

In [ ]:
from base.scene import Scene

pt = 0
mis = 1

# Instantiate a Scene object with the Camera object
scene = Scene(integrator=pt, spp=256, max_depth=6, sample_lights=1, sample_bsdf=1)

In [ ]:
from base.render import render

# Initialize the image field outside the kernel using the actual values
image = ti.Vector.field(3, dtype=ti.f32, shape=(camera[None].width, camera[None].height))


start_t = time.time()

# Call the render_scene kernel
render(scene, image, lights, camera[None], ordered_prims, linear_bvh)

end_t = time.time()

print("Elapsed (with compilation) = %s" % (end_t - start_t))

In [ ]:
image_np = image.to_numpy()
# if np.isnan(image_np).any() or np.isinf(image_np).any():
#     print("Warning: NaN or Inf values detected in the image")
# image_np = np.clip(image_np, 0, 1)  # Clip values between 0 and 1
plt.imshow(image_np)
plt.axis('off')
plt.show()

In [ ]:
def tone_mapping(hdr_image, gamma=2.2):
    ldr_image = hdr_image / (1.0 + hdr_image)
    ldr_image = np.power(ldr_image, 1.0 / gamma)
    return np.clip(ldr_image, 0, 1)

ldr_image = tone_mapping(image_np)

# Display the tone-mapped image
plt.imshow(ldr_image)
plt.axis('off')
plt.show()